In [1]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0   140k      0 --:--:-- --:--:-- --:--:--     0-- --:--:-- --:--:--  142k
Loading bbSearch Version 2.1 (at 01:48, Fri 03 Mar)
Last module source code edit 9am Thursday 24th Feb 2022


In [2]:
class Robot:
    def __init__(self, location, carried_items, strength):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength
        
    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.carried_items])
    
    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location, 
                      self.carried_items,
                      self.strength ) )
            
class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}
        
    ## Define a unique string representation for a door object    
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [3]:
class State:
    def __init__( self, robot, doors, room_contents ):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents
        
    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of 
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )
  

In [4]:
class SearchProblem:

    def __init__( self ):
        """
        The __init__ method must set the initial state for the search.
        Arguments could be added to __init__ and used to configure the
        initial state and/or other aspects of a problem instance.
        """
        self.initial_state = None ## Change this to set the actual initial state!
        raise NotImplementedError

    def info(self):
        """
        This function is called when the search is stared and should
        print out useful information about the problem.
        """
        print("This is the general SearchProblem parent class")
        print("You must extend this class to encode a particular search problem.")

    def possible_actions(self, state):
        """
        This takes a state as argument and must return a list of actions.
        Both states and actions can be any kinds of python data type (e.g.
        numbers, strings, tuples or complex objects of any class).
        """
        return []

    def successor(self, state, action):
        """
        This takes a state and an action and returns the new state resulting
        from doing that action in that state. You can assume that the given 
        action is in the list of 'possible_actions' for that state. 
        """
        return state

    def goal_test(self, state):
        """
        This method should return True or False given any state. It should return 
        True for all and only those states that are considert "goal" states.
        """
        return False

    def cost(self, path, state):
        """
        This is an optional method that you only need to define if you are using
        a cost based algorithm such as "uniform cost" or "A*". It should return
        the cost of reaching a given state via a given path.
        If this is not defined, it will is assumed that each action costs one unit
        of effort to perform, so it returns the length of the path.
        """
        return len(path)

    def heuristic(self, state):
        """
        This is an optional method that should return a heuristic value for any
        state. The value should be an estimate of the remaining cost that will be
        required to reach a goal. For an "admissible" heuristic, the value should
        always be equal to or less than the actual cost.
        """
        raise NotImplementedError

    def display_action(self, action):
        """
        You can set the way an action will be displayed in outputs.
        """
        print(action)

    def display_state(self, state):
        """
        You can set the way a state will be displayed in outputs.
        """
        print(state)

    def display_state_path( self, actions ):
        """
        This defines output of a solution path when a list of actions
        is applied to the initial state. It assumes it is a valid path
        with all actions being possible in the preceeding state.
        You probably don't need to override this.
        """
        s = self.initial_state
        self.display_state(s)
        for a in actions:
            self.display_action(a)
            s = self.successor(s,a)
            self.display_state(s)

In [5]:
class Robot:
    def __init__(self, location, carried_items, strength):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength
        
    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.carried_items])
    
    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location, 
                      self.carried_items,
                      self.strength ) )
            
class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}
        
    ## Define a unique string representation for a door object    
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

from copy import deepcopy

class RobotWorker( SearchProblem ):
    
    def __init__( self, state, goal_item_locations ):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations
        
    def possible_actions( self, state ):
        
        robot_location = state.robot.location
        strength       = state.robot.strength
        weight_carried = state.robot.weight_carried()
        
        actions = []

        
        for doorIndex, door in enumerate(state.doors):
            if door.locked==True and robot_location in door.goes_between and door.doorkey in state.robot.carried_items:
                actions.append( ("unlock", doorIndex) )

        for i in state.robot.carried_items:
            # if the current state is R0, allow it to put down the item
            if robot_location == 'R0':
                actions.append( ("put down", i) )
        # If there is a door that is locked and the robot is in the room and the robot has the key to that door


        # Can pick up any item in room if strong enough    
        for i in state.room_contents[robot_location]:
                actions.append( ("pick up", i))
                
        # If there is an unlocked door between robot location and
        # another location can move to that location
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append( ("move to", door.other_loc[robot_location]) )
                
        # Now the actions list should contain all possible actions
        return actions
    
    def successor( self, state, action):
        next_state = deepcopy(state)
        act, target = action
        
        if act== "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)  

        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)
            
        if act == "move to":
            next_state.robot.location = target

        if act == "unlock":
            # at index target[0], modify the door to be unlocked in next_state
            next_state.doors[target].locked = False
            
                

                    
                    
            
        
            
        return next_state
        
    def goal_test(self, state):
        #print(state.room_contents)
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
        return True
    
    def countItemsHeuristic(self, state):
        # This is a very simple heuristic that just counts the number of items 
        # that are not in their goal locations or not in possession of the robot. It is admissible because it
        # never overestimates the cost of reaching a goal.
        count = 0
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    count += 1
        return count

    def estimateDistanceHeuristic(self, state):

        # A more complex heuristic that tries to estimate the cost of reaching a goal by
        # considering the distance to the nearest goal item from the robot's current location.
        # It is admissible because it never overestimates the cost of reaching a goal.
        # It is not consistent because it does not always return a value that is less than
        # the cost of reaching a goal from the current state plus the cost of the action.
        
        # First, find the nearest goal item to the robot's current location
        min_dist = 1000000
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    robotLocation = state.robot.location
                    # calculate what number the room is
                    roomNumber = int(room[1:])
                    global distance
                    dist = distance[room]
                    dist = dist[roomNumber]
                    if dist < min_dist:
                        min_dist = dist
        # Now add the distance to the nearest goal item from the robot's current location
        # to the number of items the robot is carrying
        return min_dist + len(state.robot.carried_items)

    def display_state(self,state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)

In [6]:
# ROOM_CONTENTS = {
#     'R0'     : set(),
#     'R1'     : set(),
#     'R2'     : {'D4 key', 'D5 key', 'white coin'},
#     'R3'     : {'silver coin'},
#     'R4'     : {'copper coin', 'brown coin', 'grey coin'},
#     'R5'     : {'D8 key'}
# }

ROOM_CONTENTS = {
    'R0'     : set(),
    'R1'     : set(),
    'R2'     : {'D4 key'},
    'R3'     : {'D5 key'},
    'R4'     : set(),
    'R5'     : {'D8 key'}
}

ITEM_WEIGHT = {
    'D4 key' : 0,
    'D5 key' : 0,
    'D8 key' : 0,
    'D4 key' : 0,
    'gold coin' : 0,
    'silver coin' : 0,
    'copper coin' : 0,
    'brown coin' : 0,
    'grey coin' : 0,
    'white coin' : 0
}
          
DOORS = [
    Door( 'R0', 'R1', locked=False ),
    Door( 'R1', 'R2', locked=False ),
    Door( 'R1', 'R3', locked=False ),
    Door( 'R1', 'R4', locked=False ),
    Door( 'R2', 'R3', doorkey='D4 key', locked=True),
    Door( 'R4', 'R5', locked=False ),
    Door( 'R3', 'R5', doorkey='D5 key', locked=True),
    Door( 'R4', 'R0', doorkey='D8 key', locked=True)
]    
    
distance = {"R0":[0, 1, 2, 2, 1, 2],
            "R1":[1, 0, 1, 1, 1, 2],
            "R2":[2, 1, 0, 1, 2, 2],
            "R3":[2, 1, 1, 0, 2, 1],
            "R4":[1, 1, 2, 2, 0, 1],
            "R5":[2, 2, 2, 1, 1, 0]}


In [7]:
algorithms = [
    {
        'name': 'Breadth First Search',
        'mode': "BF/FIFO",
        'cost': None,
        'randomise': False,
        'heuristic': None,
        'results': [],
    },
    {
        'name': 'Depth First Search',
        'mode': "DF/LIFO",
        'cost': None,
        'randomise': False,
        'heuristic': None,
        'results': [],
    },
    {
        'name': 'Depth First Search (Randomised)',
        'mode': "DF/LIFO",
        'cost': None,
        'randomise': True,
        'heuristic': None,
        'results': [],
    },
    {
        'name': 'Greedy Search (estimateDistanceHeuristic)',
        'mode': "BF/FIFO",
        'cost': None,
        'randomise': False,
        'heuristic': 'estimateDistanceHeuristic', 'results': [],
    },
    {
        'name': 'A* Search (estimateDistanceHeuristic)',
        'mode': "BF/FIFO",
        'cost': True,
        'randomise': False,
        'heuristic': 'estimateDistanceHeuristic',
        'results': [],
    },
    {
        'name': 'Greedy Search (countItemsHeuristic)',
        'mode': "BF/FIFO",
        'cost': None,
        'randomise': False,
        'heuristic': 'countItemsHeuristic', 'results': [],
    },
    {
        'name': 'A* Search (countItemsHeuristic)',
        'mode': "BF/FIFO",
        'cost': True,
        'randomise': False,
        'heuristic': 'countItemsHeuristic',
        'results': [],
    },
]

max_nodes = 100000

rob = Robot('R0', [], 100)
state = State(rob, DOORS, ROOM_CONTENTS)
goal_item_locations =  {"R0":{"D4 key", "D5 key", "D8 key"}} 

for algorithm in algorithms:
    RW_PROBLEM_1 = RobotWorker( state, goal_item_locations )  
    result = search(RW_PROBLEM_1,
                    mode=algorithm['mode'],
                    max_nodes=max_nodes,
                    loop_check=True,
                    return_info=True,
                    cost= RW_PROBLEM_1.cost if algorithm['cost'] else None,
                    heuristic=RW_PROBLEM_1.estimateDistanceHeuristic if algorithm['heuristic'] == "estimateDistanceHeuristic" else RW_PROBLEM_1.countItemsHeuristic if algorithm['heuristic'] == "countItemsHeuristic" else None,
                    randomise=algorithm['randomise']
                    )
    algorithm['results'].append(result)



This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 15
Goal state is:
Robot location: R0
Robot carrying: []
Room contents: {'R0': {'D8 key', 'D4 key', 'D5 key'}, 'R1': set(), 'R2': set(), 'R3': set(), 'R4': set(), 'R5': set()}
The action path to the solution is:
('move to', 'R1')
('move to', 'R2')
('pick up', 'D4 key')
('unlock', 4)
('move to', 'R3')
('pick up', 'D5 key')
('unlock', 6)
('move to', 'R5')
('pick up', 'D8 key')
('move to', 'R4')
('unlock', 7)
('move to', 'R0')
('put down', 'D4 key')
('put down', 'D5 key')
('put down', 'D8 key')


SEARCH SPACE STATS:
Total nodes generated          =     2436  (includes start)
Nodes discarded by loop_check  =     1412  (1024 distinct states added to queu

In [8]:
print("Algorithm | Nodes Generated | Nodes tested | Nodes Discarded | Distinct States Seen | Nodes Left in Queue | Time Taken | Path Length \n------|-----|-----|-----|-----|-----|-----|-----|")

index = 0
for algorithm in algorithms:
    print(algorithm['name'], end="")
    index2 = 0
    for result in algorithm['results']:
        print(" | " + str(result['search_stats']['nodes_generated']) + " | " + str(result['search_stats']['nodes_tested']) + " | " + str(result['search_stats']['nodes_discarded']) + " | " + str(result['search_stats']['distinct_states_seen']) + " | " + str(result['search_stats']['nodes_left_in_queue']) + " | " + str(result['search_stats']['time_taken']) + " | " + str(result['result']['path_length']))
        index2+=1
    index+=1

Algorithm | Nodes Generated | Nodes tested | Nodes Discarded | Distinct States Seen | Nodes Left in Queue | Time Taken | Path Length 
------|-----|-----|-----|-----|-----|-----|-----|
Breadth First Search | 2436 | 823 | 1412 | 1024 | 201 | 0.42462350000000093 | 15
Depth First Search | 95 | 35 | 41 | 54 | 19 | 0.016718958999998534 | 23
Depth First Search (Randomised) | 151 | 48 | 73 | 78 | 30 | 0.02639654100000044 | 18
Greedy Search (estimateDistanceHeuristic) | 3540 | 1231 | 2294 | 1246 | 15 | 0.602342084 | 20
A* Search (estimateDistanceHeuristic) | 3540 | 1231 | 2294 | 1246 | 15 | 0.6781459160000001 | 15
Greedy Search (countItemsHeuristic) | 106 | 43 | 36 | 70 | 27 | 0.018875999999998783 | 20
A* Search (countItemsHeuristic) | 1563 | 555 | 798 | 765 | 210 | 0.28830408399999996 | 15


Algorithm | Nodes Generated | Nodes tested | Nodes Discarded | Distinct States Seen | Nodes Left in Queue | Time Taken | Path Length 
------|-----|-----|-----|-----|-----|-----|-----|
Breadth First Search | 2360 | 797 | 1382 | 978 | 181 | 0.4213824579996981 | 15
Depth First Search | 95 | 35 | 41 | 54 | 19 | 0.019528833000094892 | 23
Depth First Search (Randomised) | 71 | 27 | 25 | 46 | 19 | 0.015313999999762018 | 22
Greedy Search (estimateDistanceHeuristic) | 3344 | 1159 | 2178 | 1166 | 7 | 0.6343554589998348 | 20
A* Search (estimateDistanceHeuristic) | 3344 | 1159 | 2178 | 1166 | 7 | 0.5985580410001603 | 15
Greedy Search (countItemsHeuristic) | 106 | 43 | 36 | 70 | 27 | 0.0195568330000242 | 20
A* Search (countItemsHeuristic) | 1500 | 535 | 774 | 726 | 191 | 0.27240549999987707 | 15

Scenario:

The robot is inside a castle-like structure. The old castle contains many secret passageways that are locked. They can only be accessed by first finding a secret key and then opening the passageway. Passageways (denoted D) can be locked or unlocked. The castle contains 6 rooms (denoted R) and the robot starts in R0. Each room may or may not contain a chest (denoted C), which contains valuable items. The goal of the robot is to traverse through this castle and pick up all of the valuables from the chests, which are all keys. However, the robot should traverse the castle in such a way it performs the lowest amount of actions. Passageways D1, D2, D3, D6, and D7 are unlocked by default. Passageways D4, D5, and D8 are locked. Chest C2 contains the D4 key. C1 contains the D5 key. C3 contains the D8 key. The action of opening a chest is abstracted away and not counted as an action. As such, the robot can open the chest and pick up the key in the same step. The robot can change location from one room to another by using a passageway. The robot can unlock locked passageways if it has the correct key in its contents. The robot can pick up items from the chest if there is a chest in the current room which contains items.
The robot can put down items only in R0 since its goal is to fetch every item and place it in R0. This is done to reduce the number of possibilities the robot could go through. 


Diagram in discord: 

Heutirist: 
1) The first heuristic is quite simple. It counts how many of the goal items you currently have in your possession or currently are in the goal room. It is admissible because it never overestimates the cost of reaching a goal. 


In [9]:
def countItemsHeuristic(self, state):
    # count the number of items in the wrong room
    count = 0
    for room, contents in self.goal_item_locations.items():
        for i in contents:
            if not i in state.room_contents[room]:
                count += 1
    return count

2) The second heuristic is more complex.  It is a modified version of the Manhattan heuristic seen in the previous problem. It tries to estimate the cost of reaching a goal by considering the distance to the nearest goal item from the robot's current location. It is admissible because it never overestimates the cost of reaching a goal. However, it is not consistent because it does not always return a value that is less than the cost of reaching a goal from the current state plus the cost of the action.

In [10]:
def estimateDistanceHeuristic(self, state):
    # First, find the nearest goal item to the robot's current location
    min_dist = 1000000
    for room, contents in self.goal_item_locations.items():
        for i in contents:
            if not i in state.room_contents[room]:
                dist = self.distance(state.robot.location, room)
                if dist < min_dist:
                    min_dist = dist
    # Now add the distance to the nearest goal item from the robot's current location
    # to the number of items the robot is carrying
    return min_dist + len(state.robot.carried_items)
